In [83]:
%scala
import org.apache.spark.sql.types.{StructType, StructField, StringType, IntegerType,DoubleType};
import org.apache.spark.sql.SparkSession
import java.io.File
import org.apache.spark.SparkContext
import org.apache.spark.sql._
import scala.collection.mutable

import org.apache.spark.sql.types.{StructType, StructField, StringType, IntegerType, DoubleType}
import org.apache.spark.sql.SparkSession
import java.io.File
import org.apache.spark.SparkContext
import org.apache.spark.sql._
import scala.collection.mutable


In [84]:
 val spark = SparkSession
 .builder
 .appName("GoodReads")
 .getOrCreate()

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@7dc0dd3f


In [85]:
val schema = StructType(Array(StructField("Id", IntegerType, false),
                          StructField("Name", StringType, true),
                          StructField("Authors", StringType, true),
                          StructField("PagesNumber", IntegerType, true),
                          StructField("RatingDist1", StringType, true),
                          StructField("RatingDist2", StringType, true),
                          StructField("RatingDist3", StringType, true),
                          StructField("RatingDist4", StringType, true),
                          StructField("RatingDist5", StringType, true),
                          StructField("RatingDistTotal", StringType, true),
                          StructField("PublishDay", StringType, true),
                          StructField("PublishMonth", StringType, true),
                          StructField("PublishYear", StringType, true),
                          StructField("Publisher", StringType, true),
                          StructField("CountsOfReview", IntegerType, true),
                          StructField("Language", StringType, true),
                          StructField("Rating", DoubleType, true),
                          StructField("ISBN", StringType, true)))

schema: org.apache.spark.sql.types.StructType = StructType(StructField(Id,IntegerType,false), StructField(Name,StringType,true), StructField(Authors,StringType,true), StructField(PagesNumber,IntegerType,true), StructField(RatingDist1,StringType,true), StructField(RatingDist2,StringType,true), StructField(RatingDist3,StringType,true), StructField(RatingDist4,StringType,true), StructField(RatingDist5,StringType,true), StructField(RatingDistTotal,StringType,true), StructField(PublishDay,StringType,true), StructField(PublishMonth,StringType,true), StructField(PublishYear,StringType,true), StructField(Publisher,StringType,true), StructField(CountsOfReview,IntegerType,true), StructField(Language,StringType,true), StructField(Rating,DoubleType,true), StructField(ISBN,StringType,true))


In [86]:
def getListOfFiles(dir: String):List[File] = {
    val d = new File(dir)
    if (d.exists && d.isDirectory) {
        d.listFiles.filter(_.isFile).toList
    } else {
        List[File]()
    }
}

val books_list = getListOfFiles("books")

getListOfFiles: (dir: String)List[java.io.File]
books_list: List[java.io.File] = List(books/book1-100k.csv, books/book100k-200k.csv, books/book1400k-1500k.csv, books/book200k-300k.csv, books/book300k-400k.csv, books/book400k-500k.csv, books/book500k-600k.csv, books/user_rating_0_to_1000.csv, books/user_rating_1000_to_2000.csv, books/user_rating_2000_to_3000.csv, books/user_rating_3000_to_4000.csv, books/user_rating_4000_to_5000.csv, books/user_rating_5000_to_6000.csv, books/user_rating_6000_to_11000.csv)


In [123]:
val dataframes_list = mutable.ArrayBuffer[DataFrame]()

for (b<-books_list){

    dataframes_list+=spark.read.format("csv") 
            .option("header", "true")    
            .option("quote", "\"")
            .option("escape", "\"")
            .option("ignoreLeadingWhiteSpace", "true")
            .option("ignoreTrailingWhiteSpace", "true")
            .option("multiLine", "true")
            .load(b.toString)
}

dataframes_list: scala.collection.mutable.ArrayBuffer[org.apache.spark.sql.DataFrame] = ArrayBuffer([Id: string, Name: string ... 16 more fields], [pagesNumber: string, Authors: string ... 16 more fields], [Id: string, Name: string ... 18 more fields], [Publisher: string, RatingDistTotal: string ... 16 more fields], [RatingDist4: string, RatingDist1: string ... 16 more fields], [PublishYear: string, Rating: string ... 16 more fields], [Id: string, Name: string ... 16 more fields], [ID: string, Name: string ... 1 more field], [ID: string, Name: string ... 1 more field], [ID: string, Name: string ... 1 more field], [ID: string, Name: string ... 1 more field], [ID: string, Name: string ... 1 more field], [ID: string, Name: string ... 1 more field], [ID: string, Name: string ... 1 more field])


In [163]:
var dataframe = spark.createDataFrame(spark.sparkContext.emptyRDD[Row], schema)

dataframe: org.apache.spark.sql.DataFrame = [Id: int, Name: string ... 16 more fields]


In [185]:
for (f<-dataframes_list){
 dataframe = dataframe.unionByName(f, allowMissingColumns = true)
}

In [182]:
dataframe.select("Name","Rating").orderBy(desc("Rating")).show(15, true)

+--------------------+---------------+
|                Name|         Rating|
+--------------------+---------------+
|             The BFG|really liked it|
|White Hunters: Th...|really liked it|
|Ready-to-Use Old-...|really liked it|
|       The Rails Way|really liked it|
|The Chronicles of...|really liked it|
|Ready Player One ...|really liked it|
|The Religion (Tan...|really liked it|
|The Collapsing Em...|really liked it|
|          The Circle|really liked it|
|Thunderball (Jame...|really liked it|
|    The Anubis Gates|really liked it|
|La Belle Sauvage ...|really liked it|
|The Complete Sher...|really liked it|
|Moon Handbooks So...|really liked it|
|         Street Boys|really liked it|
+--------------------+---------------+
only showing top 15 rows



In [172]:
dataframe.agg(mean("Rating").alias("Rating promedio")).show()

dataframe.groupBy("Authors").agg(mean("Rating").alias("Rating promedio")).show()

dataframe.groupBy("Publisher").agg(mean("Rating").alias("Rating promedio")).show()

dataframe.agg(mean("PagesNumber").alias("Numero de páginas promedio de todos los libros")).show()

dataframe.groupBy("Authors").agg(mean("PagesNumber").alias("Numero de páginas promedio de todos los libros")).show()

dataframe.groupBy("Publisher").agg(mean("PagesNumber").alias("Numero de páginas promedio de todos los libros")).show()

dataframe.groupBy("Authors").agg(count("Id").alias("Número de libros publicados")).show()

+-----------------+
|  Rating promedio|
+-----------------+
|3.464592991292083|
+-----------------+

+--------------------+------------------+
|             Authors|   Rating promedio|
+--------------------+------------------+
|Michael Eliot Howard| 3.768333333333333|
|           Neal Ford|              3.38|
|       Rachel Andrew|           3.37875|
|          James Frey| 3.728333333333333|
|         Bill Bright|             4.128|
|     Eric Klinenberg|              3.63|
|     Karen Armstrong|3.9354285714285715|
|          Dava Sobel|          3.779375|
|        Richard King|              3.98|
|        Joseph Monti|               4.0|
|        Philip Hofer|               0.0|
|         Ann Beattie|              3.56|
|          Jack Ahern|               2.5|
|        Steve Kokker|              3.92|
|        Diane Conway|              1.79|
|       Julian Harvey|               0.0|
|     Thomas S. Mowle|               0.0|
|        Helena Grice|             4.225|
|        Sarah Le